# NMSI Operators Demo — `e` and `π*`

This notebook provides a lightweight, NumPy-only demonstration of the **exponential operator `e`** and the **cyclic forcing `π*`** used in the NMSI–π*–HDQG–e framework.

We evolve a synthetic 2D vector field under four scenarios:
1) **Baseline** (no extra terms),
2) **`e` only** (exponential damping),
3) **`π*` only** (bounded zero-mean actuation),
4) **`π* + e`** (combined).

We track **Energy** $E(t)=\tfrac12\langle u^2+v^2\rangle$ and **Enstrophy** $\Omega(t)=\langle \omega^2\rangle$ (with a simple FD curl) to visualize stabilization.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def e_operator_factor(t, lam_e=0.6, alpha_e=0.25):
    return -lam_e * np.exp(-alpha_e * t)

def e_term(u, t, lam_e=0.6, alpha_e=0.25):
    return e_operator_factor(t, lam_e, alpha_e) * u

def pi_star_factor(t, A_pi=0.15, omega_pi=3.5):
    return A_pi * np.sin(omega_pi * t)

def pi_star_term(u, t, A_pi=0.15, omega_pi=3.5):
    return pi_star_factor(t, A_pi, omega_pi) * u

def energy(u, v):
    return 0.5*np.mean(u*u + v*v)

def enstrophy(u, v):
    vx = np.roll(v, -1, axis=1) - np.roll(v, 1, axis=1)
    uy = np.roll(u, -1, axis=0) - np.roll(u, 1, axis=0)
    w  = vx - uy
    return np.mean(w*w)

## Synthetic evolution model (toy ODE)
We integrate a simple linear ODE per component:
$$ \partial_t u = a\,u + b(t)\,u, $$
where `a=0` (baseline drift=0), `b(t)` comes from `e` (damping) and/or `π*` (bounded zero-mean). This isolates their effects.

In [2]:
nx, ny = 128, 128
rng = np.random.default_rng(0)
u0 = rng.standard_normal((nx, ny))*0.1
v0 = rng.standard_normal((nx, ny))*0.1

dt, T = 5e-3, 6.0
steps = int(T/dt)

lam_e, alpha_e = 0.6, 0.25
A_pi, omega_pi = 0.15, 3.5

def run(mode):
    u = u0.copy(); v = v0.copy(); t=0.0
    E, Z, TT = [], [], []
    for k in range(steps):
        fac = 0.0
        if 'e' in mode:
            fac += e_operator_factor(t, lam_e, alpha_e)
        if 'pi' in mode:
            fac += pi_star_factor(t, A_pi, omega_pi)
        # forward Euler (sufficient for linear toy ODE)
        u = u + dt * (fac * u)
        v = v + dt * (fac * v)
        t += dt
        if k % 10 == 0:
            E.append(energy(u, v)); Z.append(enstrophy(u, v)); TT.append(t)
    return np.array(TT), np.array(E), np.array(Z)

T0,E0,Z0   = run(mode=set())          # baseline
T1,E1,Z1   = run(mode={'e'})          # e only
T2,E2,Z2   = run(mode={'pi'})         # pi* only
T3,E3,Z3   = run(mode={'e','pi'})     # pi* + e

In [3]:
fig, ax = plt.subplots(1,2, figsize=(12,4.2))
ax[0].plot(T0,E0,label='Baseline')
ax[0].plot(T1,E1,label='e')
ax[0].plot(T2,E2,label='π*')
ax[0].plot(T3,E3,label='π* + e')
ax[0].set_title('Energy E(t)'); ax[0].set_xlabel('t'); ax[0].set_ylabel('E'); ax[0].grid(True); ax[0].legend()

ax[1].plot(T0,Z0,label='Baseline')
ax[1].plot(T1,Z1,label='e')
ax[1].plot(T2,Z2,label='π*')
ax[1].plot(T3,Z3,label='π* + e')
ax[1].set_title('Enstrophy Ω(t)'); ax[1].set_xlabel('t'); ax[1].set_ylabel('Ω'); ax[1].grid(True); ax[1].legend()
plt.tight_layout(); plt.show()